In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是由[Fabian Hirschmann](https://github.com/fhirschmann)和[Rajesh Thallam](https://github.com/RajeshThallam)贡献的笔记本的更新版本。

使用R内核的Vertex AI超参数调整

<table align="left">

  <td>
    <a href="https://colab.sandbox.google.com/github/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
<a href=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_r_kernel.ipynb target='_blank'>      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看在GitHub中的代码
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_r_kernel.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                        
</table>

## 概述

此示例演示了如何使用 R 写的模型在 `Vertex AI` 上进行超参数调整，使用 R 内核，比如在 `Vertex AI Workbench Notebooks` 中。

### 目标

在本教程中，您将学习如何使用R内核来调整R自定义模型的超参数。

本教程使用以下Google Cloud ML服务：

- `Vertex AI训练`
- `Vertex AI预测`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 创建自定义的R训练脚本
- 创建自定义的R部署容器。
- 使用`Vertex AI`进行超参数调整。

### 数据集

本教程使用的数据集是[加州住房数据集](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html)。该数据集包含来自1990年加州人口普查的信息。数据集可以公开访问Cloud Storage上的`gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv`地址。该数据集用于训练随机森林回归器，以预测给定经度、纬度以及来自相应人口普查区块组的数据的中位房价。区块组是美国人口普查局发布样本数据的最小地理单元（一个区块组通常有600到3,000人口）。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

### 设置本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。 您可以跳过这一步。

否则，请确保您的环境符合本笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* R
* Python 3
* virtualenv
* 在具有R和Python 3的虚拟环境中运行的Jupyter笔记本

Cloud Storage指南[设置Python开发环境](https://cloud.google.com/python/setup) 和[Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一组简要的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装R]()。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用R的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端shell的命令行中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端shell的命令行中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

#### 为此笔记本定义辅助函数

首先，您定义了一些在整个教程中使用的辅助函数。

- `sh`：执行指定的命令shell。
- `display_file`：显示指定文件的内容。

In [ ]:
library(glue)
library(IRdisplay)

sh <- function(cmd, args = c(), intern = FALSE) {
    if (is.null(args)) {
        cmd <- glue(cmd)
        s <- strsplit(cmd, " ")[[1]]
        cmd <- s[1]
        args <- s[2:length(s)]
    }
    ret <- system2(cmd, args, stdout = TRUE, stderr = TRUE)
    if ("errmsg" %in% attributes(attributes(ret))$names) cat(attr(ret, "errmsg"), "\n")
    if (intern) return(ret) else cat(paste(ret, collapse = "\n"))
}

display_file <- function(filename) {
    body <- sh(glue("pygmentize -g {filename} -f html -P full -O style=default"), intern = TRUE)
    IRdisplay::display_html(paste("<div style='text-align: left'>", paste(body, collapse="\n"), "</div>", collapse="\n"))
}

## 安装

安装以下包以执行此笔记本。

In [ ]:
required_packages < -c("reticulate", "glue", "httr")
install.packages(setdiff(required_packages, rownames(installed.packages())))

sh("pip install --upgrade google-cloud-aiplatform")

在你开始之前

### 设置您的 Google 云项目

**无论您使用哪种笔记本环境，以下步骤均为必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得 $300 的免费信用额度，可用于支付计算/存储成本。

1. 确保您的项目已启用计费功能，详情请参阅 [启用项目的计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. 启用 Vertex AI API 和 [Artifact Registry API](https://console.cloud.google.com/flows/enableapi?apiid=artifactregistry.googleapis.com)。

1. 如果您正在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 让项目 ID 自动检测或在下方输入。然后运行单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID < -"[your-project-id]"  # @param {type:"string"}

In [ ]:
# Get your Google Cloud project ID from gcloud
if (Sys.getenv("IS_TESTING") == "") {
    if (PROJECT_ID == "[your-project-id]") {
      PROJECT_ID <- sh("gcloud config list --format 'value(core.project)' 2>/dev/null", intern = TRUE)
    }
    cat("Project ID:", PROJECT_ID, "\n")
}

In [ ]:
sh("gcloud config set project {PROJECT_ID}")

区域

您也可以更改“REGION”变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在Vertex AI上使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION <- "[your-region]"  # @param {type: "string"}

if (REGION == "[your-region]") {
    REGION <- "us-central1"
}

时间戳

如果您处于现场教程会话中，您可能正在使用共享的测试账户或项目。为了避免在创建的资源上的用户之间的名称冲突，您为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
TIMESTAMP = format(Sys.time(), "%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI工作台笔记本**，您的环境已经通过验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照要求进行身份验证以通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务账户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账户**。

3. 在**服务账户名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务账户对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

5. 点击*创建*。包含您密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中，将您的服务账户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
# The Google Cloud Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK <- file.exists("/opt/deeplearning/metadata/env_version")
IS_GOOGLE_COLAB <- (system("python -c 'import google.colab'") != 0)

GOOGLE_APPLICATION_CREDENTIALS <- ""  # @param {type:"string"}

# If on Vertex AI Workbench Notebooks, then don't execute this code
if (!IS_WORKBENCH_NOTEBOOK) {
    # Replace the string below with the path to your service account key and run this cell
    # to authenticate your GCP account.
    Sys.setenv("GOOGLE_APPLICATION_CREDENTIALS" = GOOGLE_APPLICATION_CREDENTIALS)
}

### 创建一个云存储桶

**无论您使用的是什么笔记本环境，以下步骤都是必需的。**

在初始化用于Python的Vertex AI SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在不同会话中保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME < -"[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI < -paste0("gs://", BUCKET_NAME)

In [ ]:
if (BUCKET_NAME == "" || BUCKET_NAME == "[your-bucket-name]") {
    BUCKET_NAME = paste0(PROJECT_ID, "aip-", TIMESTAMP)
    BUCKET_URI = paste0("gs://", BUCKET_NAME)
}

BUCKET_URI

只有在您的存储桶尚未存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
sh("gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}")

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
sh("gsutil ls -al {BUCKET_URI}")

### 导入库并定义常量

导入并初始化 `reticulate` R 包，以便与使用 Python 编写的 Vertex AI SDK 进行接口交互。

In [ ]:
library(reticulate)
library(glue)
library(httr)
library(IRdisplay)

use_python(Sys.which("python3"))

aiplatform <- import("google.cloud.aiplatform")

### 初始化Python版本的Vertex AI SDK

为您的项目和相应的存储桶初始化Python版本的Vertex AI SDK。

In [ ]:
aiplatform$init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_URI)

## Google Artifact Registry简介

`Google Artifact Registry`是一个用于存储和管理私有存储库中的构件的服务，包括容器镜像、Helm图表和语言包。它是Google Cloud推荐的容器镜像注册表。

了解更多关于[Docker快速入门](https://cloud.google.com/artifact-registry/docs/docker/quickstart)。

### 启用 Artifact Registry API

首先，您必须为您的项目启用 Artifact Registry API 服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
sh("gcloud services enable artifactregistry.googleapis.com")

创建一个私有的Docker存储库

您的第一步是在Google Artifact Registry中创建自己的Docker存储库。

1. 运行`gcloud artifacts repositories create`命令，创建一个带有描述“docker存储库”的新Docker存储库。

2. 运行`gcloud artifacts repositories list`命令，验证您的存储库是否已创建。

In [ ]:
PRIVATE_REPO < -"my-docker-repo"

sh(
    'gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"'
)

sh("gcloud artifacts repositories list")

### 配置身份验证到您的私有存储库

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具对您所在地区的`Artifact Registry`进行身份验证请求。

In [ ]:
sh("gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet")

创建Dockerfile

您的定制容器的Docker文件是基于深度学习容器构建的 - 这个容器也被用于Vertex AI Workbench。此外，您还添加了一个用于模型训练的R脚本。

In [ ]:
IMAGE_NAME < -"vertex-r"  # @param {type:"string"}
IMAGE_TAG < -"latest"  # @param {type:"string"}
IMAGE_URI < -glue(
    "{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"
)

dir.create("src", showWarnings=FALSE)

In [ ]:
Dockerfile = cat("
# filename: Dockerfile - container specifications for using R in Vertex AI
FROM gcr.io/deeplearning-platform-release/r-cpu.4-1:latest

WORKDIR /root

COPY train.R /root/train.R

# Install Fortran
RUN apt-get update
RUN apt-get install gfortran -yy

# Install R packages
RUN Rscript -e \"install.packages('plumber')\"
RUN Rscript -e \"install.packages('argparser')\"
RUN Rscript -e \"install.packages('gbm')\"
RUN Rscript -e \"install.packages('caret')\"
RUN Rscript -e \"install.packages('reticulate')\"

RUN pip install cloudml-hypertune

EXPOSE 8080
", file = "src/Dockerfile")

display_file("src/Dockerfile")

### 创建训练脚本

接下来，创建文件`train.R`，用于训练您的 R 模型。该脚本将在加利福尼亚房屋数据集上训练一个`gbm`模型。Vertex AI 设置环境变量供您使用，并且每次试验的超参数作为命令行参数传递。然后将训练好的模型结果存储到您的 Cloud Storage 存储桶中。您的训练脚本的结果将使用[hypertune](https://github.com/GoogleCloudPlatform/cloudml-hypertune)包传递回给 Vertex AI，该包将一个 JSON 文件存储到`/tmp/hypertune/output.metrics`。Vertex AI使用该信息为下一个试验制定超参数配置，以及评估哪个试验对于整体最佳结果负责。

In [ ]:
cat('
#!/usr/bin/env Rscript
# filename: train.R - perform hyperparamter tuning on a boosted tree model using Vertex AI

Sys.getenv()
library(tidyverse)
library(data.table)
library(argparser)
library(jsonlite)
library(reticulate)
library(caret)

# The GCP Project ID
project_id <- Sys.getenv("CLOUD_ML_PROJECT_ID")

# The GCP Region
location <- Sys.getenv("CLOUD_ML_REGION")

# The Cloud Storage URI to upload the trained model artifact to
model_dir <- Sys.getenv("AIP_MODEL_DIR")

# The trial ID
trial_id <- Sys.getenv("CLOUD_ML_TRIAL_ID", 0)

# The JSON file to save metric results to
#metric_file <- Sys.getenv("CLOUD_ML_HP_METRIC_FILE")
metric_file <- "/var/hypertune/output.metrics"

# Hyperparameters for this trial
p <- arg_parser("California Housing Model") %>%
     add_argument("--n.trees", default = "100", help = "number of trees to fit", type = "integer") %>%
     add_argument("--interaction.depth", default = 3, help = "maximum depth of each tree") %>%
     add_argument("--n.minobsinnode", default = 10, help = "minimun number of observations in terminal node") %>%
     add_argument("--shrinkage", default = 0.1, help = "learning rate") %>%
     add_argument("--data", help = "path to the training data in GCS")

dir.create("/tmp/hypertune")
argv <- parse_args(p, unlist(strsplit(commandArgs(trailingOnly = TRUE), "=")))

system2("gsutil", c("cp", argv$data, "./data.csv"))
data <- fread("data.csv")
print(data)

print("Starting Model Training")
tuneGrid <- expand.grid(
    interaction.depth = as.integer(argv$interaction.depth),
    n.trees = as.integer(argv$n.trees),
    n.minobsinnode = as.integer(argv$n.minobsinnode),
    shrinkage = as.numeric(0.1)
)
print(tuneGrid)
fitControl <- trainControl(method = "cv", number = 3)
set.seed(42)
fit <- train(median_house_value ~ .,
             method = "gbm",
             trControl = fitControl,
             tuneGrid = tuneGrid,
             metric = "MAE",
             data = data
)

mean_absolute_error <- mean(fit$resample$MAE)
cat(paste("mean absolute error:", mean_absolute_error, "\\n"))

hypertune <- import("hypertune")
hpt = hypertune$HyperTune()
hpt$report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag = "mean_absolute_error",
    metric_value = as.numeric(mean_absolute_error),
    global_step = 1000)

saveRDS(fit$finalModel, "gbm.rds")
system2("gsutil", c("cp", "gbm.rds", model_dir))
', file = "src/train.R")

display_file("src/train.R")

### 构建 Docker 容器

接下来，您在 Cloud Build 上构建 Docker 容器镜像 -- 这是一个无服务器的 CI/CD 平台。

*注意:* 构建 Docker 容器镜像可能需要 10 到 15 分钟。

In [ ]:
sh("cd src && gcloud builds submit --region={REGION} --tag={IMAGE_URI} --timeout=1h")

### 定义并提交超参数调整任务

一旦您的训练应用程序被容器化，您可以定义调整任务的机器规格。在这个例子中，您可以使用 `n1-standard-4` 实例。

In [ ]:
worker_pool_specs <- list(
    list(
        'machine_spec' = list(
            'accelerator_count' = as.integer(0),
            'machine_type' = 'n1-standard-4'
        ),
        'container_spec' = list(
            "image_uri" = IMAGE_URI,
            "command" = c("Rscript", "train.R"),
            "args" = list("--data", "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv")
        ),
        'replica_count' = 1
    )
)

这个规范然后被用在一个`CustomJob`中。

In [ ]:
MODEL_DIR <- glue("{BUCKET_URI}/aiplatform-custom-job-{TIMESTAMP}")
custom_job <- aiplatform$CustomJob(
    display_name = "california-custom-job",
    worker_pool_specs = worker_pool_specs,
    base_output_dir = MODEL_DIR
)

超参数调整工作搜索最佳的超参数组合，以优化您的指标。超参数调整工作通过使用不同的超参数集运行多个训练应用程序试验来实现这一点。

您可以通过以下方式控制作业：

- `max_trial_count`：决定要允许服务运行多少试验。增加试验的数量通常会产生更好的结果，但并非总是如此。通常，经过增加试验数量后会出现边际回报递减的情况，进一步的试验对准确性几乎没有影响。在开始大量试验之前，您可能想要从少量试验开始，以了解您选择的超参数对模型准确性的影响。为了充分利用超参数调整，您不应该将最大值设置得低于您使用的超参数数量的十倍。
- `parallel_trial_count`：您可以指定有多少试验可以并行运行。并行运行试验的好处是减少训练作业需要的时间（实际时间——通常总处理时间不会改变）。但是，并行运行可能会降低调整作业的有效性。这是因为超参数调整使用之前试验的结果来指导对后续试验的超参数赋值。在并行运行时，一些试验开始时没有任何仍在运行的试验的结果的影响。

此外，您还需要指定要调整哪些超参数。很难提供有关如何选择应该调整哪些超参数的通用建议。如果您对正在使用的机器学习技术有经验，您可能对其超参数的行为有所了解。您还可以从机器学习社区中获取建议。

无论您如何选择，都很重要了解其影响。您选择要调整的每个超参数都有可能增加成功调整作业所需试验的数量。当您在 Vertex AI 上运行超参数调整作业时，您被收费的金额基于您的超参数调整作业启动的试验持续时间。慎重选择要调整的超参数可以减少超参数调整作业的时间和成本。

支持 Vertex AI 超参数调整作业的[多种数据类型]。

In [ ]:
hpt_job <- aiplatform$HyperparameterTuningJob(
    display_name = "california-hpt-job",
    custom_job = custom_job,
    max_trial_count = as.integer(14),
    parallel_trial_count = as.integer(2),
    metric_spec = list(
        "mean_absolute_error" = "minimize"
    ),
    parameter_spec = list(
        "n.trees" = aiplatform$hyperparameter_tuning$IntegerParameterSpec(
            min = as.integer(10), max = as.integer(1000), scale = "linear"
        ),
        "interaction.depth" = aiplatform$hyperparameter_tuning$IntegerParameterSpec(
            min = as.integer(1), max = as.integer(10), scale = "linear"
        ),
        "n.minobsinnode" = aiplatform$hyperparameter_tuning$IntegerParameterSpec(
            min = as.integer(1), max = as.integer(20), scale = "linear"
        )
    )
)

### 执行超参数调整作业

要训练调整模型，您需要调用方法`run()`。

In [ ]:
hpt_job$run()
hpt_job

您现在可以在Vertex AI控制台中实时观看进展。一旦作业完成，您可以看到每个试验的结果。

最后，要列出所有试验及其相应的结果，我们可以检查`hpt_job$trials`。

In [ ]:
results <- lapply(hpt_job$trials,
    function(x) { c(as.integer(x$id), as.numeric(x$final_measurement$metrics[[0]]$value)) }
)
results <- as.data.frame(do.call(rbind, results))
colnames(results) <- c("id", "metric")

In [ ]:
results

找到具有最低错误的试验。

In [ ]:
best_trial <- results[results$metric == min(results$metric), ]$id

In [ ]:
hpt_job$trials[[best_trial]]

模型文物可以在云存储中找到。

In [ ]:
sh("gsutil ls -r {MODEL_DIR}/{best_trial}")

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket <- FALSE

hpt_job$delete()

sh("rm -rf src")

# remove Docker image
sh("docker rmi {IMAGE_URI}")

if (delete_bucket == TRUE) sh("gsutil -m rm -r {BUCKET_URI}")